In [1]:
import cv2
import math
import csv
import os
import random
import numpy as np

In [2]:
def add_line_shadows(image, shadow_intensity=50, dash_width=10, dash_spacing=20, rotation=0):
    h, w, c = image.shape

    # Diagonal length
    d = math.ceil((h**2 + w**2)**0.5)

    # Mask is larger to accomodate the rotation
    mask = np.zeros((d, d, c), dtype=np.uint8)

    # Number of dashs on mask
    dash_count = d // (dash_width + dash_spacing)

    # Create dashes on the mask
    for i in range(dash_count):
        x = i * (dash_width + dash_spacing) + dash_spacing
        cv2.rectangle(mask, (x, 0), (x + dash_width, d), (shadow_intensity, shadow_intensity, shadow_intensity), -1)

    # Calculate rotation matrix using the center of the mask
    mask_center = (d // 2, d // 2)
    rotation_matrix = cv2.getRotationMatrix2D(mask_center, rotation, 1)

    # Rotate the mask
    rotated_mask = cv2.warpAffine(mask, rotation_matrix, (d, d))

    # Crop mask
    x_crop = (d - w) // 2
    y_crop = (d - h) // 2
    cropped_mask = rotated_mask[y_crop:y_crop + h, x_crop:x_crop + w]

    # Apply cropped mask
    shadowed_image = cv2.subtract(image, cropped_mask)

    return shadowed_image

In [ ]:
image = cv2.imread('Aaron_Eckhart_1.jpg')
print(image.shape)
cv2.imshow('shadowed img',add_line_shadows(image, dash_width=5, rotation=30))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
def random_parameters():
    return random.randint(30, 100), random.randint(5, 20), random.randint(10, 30)

In [8]:
def apply_shadows_and_write_csv(image_path, save_dir, csv_writer):
    # Load image
    image = cv2.imread(image_path)
    base_name = os.path.splitext(os.path.basename(image_path))[0]

    # Original image (no shadow)
    cv2.imwrite(os.path.join(save_dir, f"{base_name}_raw.jpg"), image)
    csv_writer.writerow([f"{base_name}_raw.jpg", 0, 0, 0, 0, 1])

    # Shadowed images with random parameters
    for suffix, rotation in [('_vertical', 0), ('_30', 30), ('_cross', 90)]:
        shadow_intensity, dash_width, dash_spacing = random_parameters()
        shadowed_image = add_line_shadows(image, shadow_intensity, dash_width, dash_spacing, rotation)
        if suffix == '_cross':
            shadowed_image = add_line_shadows(shadowed_image, *random_parameters(), rotation=0)

        file_name = f"{base_name}{suffix}.jpg"
        cv2.imwrite(os.path.join(save_dir, file_name), shadowed_image)
        csv_writer.writerow([file_name, shadow_intensity, dash_width, dash_spacing, rotation, 0])

In [10]:
# Directory paths
source_dir = '../benchmark/dataset/cplfw/aligned images/'
save_dir = '../benchmark/dataset/cplfw_noised/aligned_images/'
csv_file = '../benchmark/dataset/cplfw_noised/specs.csv'

random.seed(10)

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Open CSV file and write headers
with open(csv_file, mode='w', newline='') as file:
    csv_writer = csv.writer(file)
    csv_writer.writerow(['image_name', 'shadow_intensity', 'dash_width', 'dash_spacing', 'rotation', 'original'])

    # Process each image in the source directory
    for file_name in os.listdir(source_dir):
        if file_name.endswith('.jpg'):
            image_path = os.path.join(source_dir, file_name)
            apply_shadows_and_write_csv(image_path, save_dir, csv_writer)